![image-3.png](https://github.com/cafawo/Derivatives/blob/main/figures/fscampus_small.png?raw=1)

# Derivatives

**Prof. Dr. Fabian Woebbeking**</br>
Assistant Professor of Financial Economics

IWH - Leibniz Institute for Economic Research</br>
Frankfurt School of Finance & Management

fabian.woebbeking@dozent.frankfurt-school.de

## Case 3: Fixed Income

Case description ...

### Part 1

#### Task:

#### Task:

In [2]:
# Your code

### Part 2

#### Task:

#### Task: